In [ ]:
import pandas as pd
import json
import sys, os
import pycountry_convert as pc

#using gross syntax for relative import; will be fixed with packaging and bundling
# This is the relative path from the notebook to our vis extensions
example_code = '../../roundtrip/example_code/'
vis_path = os.path.join(example_code, 'magics_source')


# Add the path so that the notebook can find the Roundtrip extension
module_path = os.path.abspath(os.path.join(vis_path)) 
if module_path not in sys.path:
    sys.path.append(module_path)


%load_ext table
%load_ext scatter

# Support for Current CCT VIS Workflow in Hatchet

We begin by loading real data, downloaded from Kaggle, into a pandas dataframe and loading derived information from that dataframe.

In [ ]:
dataset = pd.read_csv(os.path.join(example_code, 'data/Electricity_Production_By_Source.csv'))
vis_in = dataset.groupby(['Entity']).max().reset_index()


def getContinent(row):
    try:
        c_code = pc.country_name_to_country_alpha2(row['Entity'], cn_name_format='default')
        return pc.country_alpha2_to_continent_code(c_code)
    except:
        return 'OTH'

vis_in['Continent'] = vis_in.apply(getContinent, axis=1)

In [ ]:
vis_in

We can use the basic functionality in roundtrip to pass this data into our scatterplot as a DataFrame (or other complex datatype). The visualization developer -- whoever built scatterplot -- will manage the necessary conversions.

In [ ]:
%scatter_plt vis_in

Once visualized, we can return our filter selections back to our notebook with a magic function provided by the visualization developer `get_filter`. This function makes a pass through call to a "return" function provided by our Roundtrip API and stores the data in `fltr`.

In [ ]:
%get_filter fltr

In [ ]:
fltr = json.loads(fltr)

`fltr` Is returned from the "get_filter" function as a jsonified string so we can load it and then filter our visualization. If we re-run our above `scatter_plt` cell we will see that the selected circles are now gone.

In [ ]:
vis_in = vis_in[~vis_in.index.isin(fltr)]

In [ ]:
vis_in